In [1]:
from sympy import *
# from sympy.plotting import plot, plot3d_parametric_line, plot3d
# from sympy.solvers.inequalities import reduce_rational_inequalities
# from sympy.stats import Poisson, Exponential, Binomial, density, moment, E, cdf
# from itertools import product

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Enable the mathjax printer
init_printing(use_latex='mathjax')

In [2]:
class Robot:
    def __init__(self, configuration:str, dh_table, name:str='Cocoabot', **matrices):
        """
        Initialize the robot

        Parameters:
        configuration: str
            Set the joint types: 'RRR', 'RRP', ...
        
        dh_table: sympy.Matrix
            Table with Denavit-Hartenberg parameters.
        
        name: str
            Define a name for the robot.
        
        matrices: sympy.Matrix
            masses:     Matrix with the mass of each link: [m1, m2, m3].T (nx1)
            r_cis:      Matrix with the com of each link (local): [ci1, ci2, ci3].T (nx1)
            inertias:   Matrix with the inertia of each link: [[I11:4], [I21:4], [I31:4]] (3x3)
            g_vec:      Matrix with the gravity vector: [0 0 -g].T (3x1)        
        """
        self.name = name
        self.joint_types = list(configuration)
        self.dof = len(self.joint_types)

        self.joints = pd.DataFrame(columns=['type', 'theta', 'd', 'a', 'alpha'])
        self.t = symbols('t')
        self.q = [Function(f'q{i+1}')(self.t) for i in range(self.dof)]
        self.dq = [qi.diff(self.t) for qi in self.q]
        self.ddq = [dqi.diff(self.t) for dqi in self.dq]
        self.set_matrices(**matrices)
        self.set_dh_params(dh_table)
        self.get_joints()
        self.compute()

    def set_matrices(self, **matrices):
        self.masses_symbols = Matrix(symbols(f'm1:{self.dof+1}'))
        self.masses = matrices.get('masses', self.masses_symbols)
        self.r_cis_symbols = [symbols(f'r_{i+1}{a}') for i in range(self.dof) for a in {'x','y', 'z'}]
        self.r_cis_local = matrices.get('r_cis', [Matrix(self.r_cis_symbols[i:i+3]) for i in range(0,3*self.dof,3)])
        self.inertias_symbols = [symbols(f'I{i+1}_1:4_1:4') for i in range(3)]
        self.inertias = matrices.get('inertias', [Matrix(3,3,self.inertias_symbols[i]) for i in range(self.dof)])
        self.g_symbol = symbols('g')
        self.g_vec = matrices.get('g_vec', Matrix([0, 0, -self.g_symbol]))
        # self.r_cis_local = matrices.get('r_cis', [Matrix([symbols(f'r_{i+1}x'), symbols(f'r_{i+1}y'), symbols(f'r_{i+1}z')]) for i in range(self.dof)])
        # self.inertias = matrices.get('inertias', [Matrix(3, 3, symbols(f'I{i+1}_11:14 I{i+1}_21:24 I{i+1}_31:34')) for i in range(self.dof)])
    
    def set_dh_params(self, dh_table):
        """
        Set the Denavit Hartenberg parameters

        Parameters:
        dh_table: list of lists
            Each list represent the dh-parameters of a joint in the order (theta, d, a, alpha)
        """
        self.thetas = dh_table[:,0]         #if dh_table else Matrix(symbols(f'theta1:{self.dof+1}'))
        self.ds = dh_table[:,1]             #if dh_table else Matrix(symbols(f'd1:{self.dof+1}'))
        self._as_ = dh_table[:,2]           #if dh_table else Matrix(symbols(f'a1:{self.dof+1}'))
        self.alphas = dh_table[:,3]         #if dh_table else Matrix(symbols(f'alpha1:{self.dof+1}'))
    
    def get_joints(self):
        q = symbols(f'q1:{self.dof+1}')
        for i, tp in enumerate(self.joint_types):
            if tp == 'R': self.thetas = self.thetas.subs(zip(q,self.q))
            if tp == 'P': self.ds = self.ds.subs(zip(q,self.q))
        self.joints = pd.DataFrame(list(zip(self.joint_types, Matrix(self.thetas), self.ds, self._as_, self.alphas)), 
                                   columns=['type', 'theta', 'd', 'a', 'alpha'], 
                                   index=np.arange(self.dof)+1)
    
    def compute(self):
        # Kynematics
        self.dh_matrices = [self.compute_dh_matrix(joint_index=i+1) for i in range(self.dof)]
        self.base_to_joint = [self.calculate_base_to_joint(joint_end=i+1) for i in range(self.dof)]
        self.base_to_end_effector = self.base_to_joint[-1]
        self.jacobian = self.calculate_jacobian()
        # Center of Mass
        self.r_cis_global = [self.calculate_r_i_ci(joint_index=i+1) for i in range(self.dof)]
        self.jacobian_ci = self.calculate_jacobian(com=True)
        # Dynamics
        self.inertia_matrix = self.calculate_inertia_matrix()
        self.coriolis_matrix = self.calculate_coriolis_matrix()
        self.gravity_vector = self.calculate_gravity_vector()
        self.torques = self.compute_inverse_dynamics()

    # def add_joint(self, _type_, dh_params=None):
    #     """
    #     Add a joint to the robot.

    #     Parameters:
    #     type (str): Type of joint ('revolute' or 'prismatic')
    #     dh_params (list): Denavit-Hartenberg parameters [theta, d, a, alpha]
    #     theta (float): Joint angle
    #     d (float): Link offset
    #     a (float): Link length
    #     alpha (float): Link twist
    #     """
    #     if dh_params is not None:
    #         theta, d, a, alpha = dh_params
    #     else:
    #         i = self.dof + 1
    #         theta, d, a, alpha = symbols(f'theta{i} d{i} a{i} alpha{i}')
        
    #     self.joints = pd.concat([pd.DataFrame([[_type_, theta, d, a, alpha]], columns=self.joints.columns, index=[self.dof]), self.joints]).sort_index()    
        
    #     self.dof = len(self.joints)
    #     self.masses = Matrix(symbols(f'm1:{self.dof+1}'))
    #     self.r_cis_local = [Matrix([symbols(f'r_{i+1}x'), symbols(f'r_{i+1}y'), symbols(f'r_{i+1}z')]) for i in range(self.dof)]
    #     self.inertias = [Matrix(3, 3, symbols(f'I{i+1}_11:14 I{i+1}_21:24 I{i+1}_31:34')) for i in range(self.dof)]

    def compute_dh_matrix(self, joint_index):
        """
        Compute the Denavit-Hartenberg transformation matrix for a given joint.

        Parameters:
        joint_index (int): Index of the joint

        Returns:
        Matrix: Denavit-Hartenberg transformation matrix
        """
        assert joint_index > 0, "Must be positive."
        i = joint_index - 1
        theta, d, a, alpha = self.thetas[i], self.ds[i], self._as_[i], self.alphas[i]
        T = Matrix([[cos(theta), -sin(theta)*cos(alpha), sin(theta)*sin(alpha), a*cos(theta)],
                     [sin(theta), cos(theta)*cos(alpha), -cos(theta)*sin(alpha), a*sin(theta)],
                     [0, sin(alpha), cos(alpha), d],
                     [0, 0, 0, 1]])
        return T
    
    def calculate_joint_to_joint(self, joint_start, joint_end):
        assert joint_start > 0, "Must be positive."
        assert joint_end >= joint_start, "End must be greater than start."
        result = eye(self.dof + 1)
        for i in range(joint_start, joint_end+1):
            result = result @ self.dh_matrices[i-1]
        return simplify(result)
    
    def calculate_base_to_joint(self, joint_end):
        return self.calculate_joint_to_joint(joint_start=1, joint_end=joint_end)
    
    def calculate_base_to_end_effector(self):
        return self.calculate_base_to_joint(joint_end=self.dof)
    
    def get_rotation(self, matrix):
        return matrix[:self.dof,:self.dof]
    
    def get_translation(self, matrix):
        return matrix[:self.dof,-1:]

    def z_i_minus_one(self, joint_index):
        result = eye(self.dof)
        for i in range(1, joint_index):
            result = result @ self.get_rotation(self.dh_matrices[i-1])
        # result = self.get_rotation(self.base_to_joint[joint_index-1])
        result = result @ Matrix([0, 0, 1])
        return simplify(result)
    
    def r_i_minus_one_to_n(self, joint_index, joint_ci=None):
        com = joint_ci is not None
        if not com:
            return self.get_translation(self.base_to_end_effector - self.base_to_joint[joint_index-1]) \
                    if joint_index > 1 else self.get_translation(self.base_to_end_effector)
        else:
            return self.r_cis_global[joint_ci-1] - self.get_translation(self.base_to_joint[joint_index - 1])

    def calculate_r_i_ci(self, joint_index):
        r_local = Matrix.vstack(self.r_cis_local[joint_index-1], Matrix([1]))
        r_ci_global = self.base_to_joint[joint_index-1] @ r_local
        return simplify(r_ci_global[:-1,:])

    def calculate_jacobian_col_i(self, joint_index, joint_ci=None):
        com = joint_ci is not None
        if com:
            if joint_ci < joint_index:
                return Matrix(6*[0])
        joint_type = self.joint_types[joint_index-1]
        if joint_type == 'R':
            J_vi = Matrix(np.cross(self.z_i_minus_one(joint_index).T, self.r_i_minus_one_to_n(joint_index, joint_ci).T)).T
            J_wi = self.z_i_minus_one(joint_index)
        elif joint_type == 'P':
            J_vi = self.z_i_minus_one(joint_index)
            J_wi = Matrix([0, 0, 0])
        J_i = Matrix(np.vstack((J_vi, J_wi)))
        return simplify(J_i)

    def calculate_jacobian(self, com=False):
        if not com:
            return Matrix(np.hstack(tuple((self.calculate_jacobian_col_i(i+1) for i in range(self.dof)))))
        else:
            return [Matrix(np.hstack(tuple((self.calculate_jacobian_col_i(j+1, i+1) 
                    for j in range(self.dof)))))
                    for i in range(self.dof)]

    def calculate_inertia_matrix(self):
        """
        Calculate the inertia matrix of the robot.

        Returns:
        sympy.Matrix
            Inertia matrix.
        """
        inertia_matrix = zeros(self.dof, self.dof)

        for i in range(self.dof):
            jv = self.jacobian_ci[i][:3, :]
            jw = self.jacobian_ci[i][3:, :]
            inertia_matrix += self.masses[i] * (jv.T @ jv) + (jw.T @ self.inertias[i] @ jw)

        return simplify(inertia_matrix)

    def calculate_coriolis_matrix(self):
        """
        Calculate the Coriolis matrix C(q), such that C(q) * dq = Coriolis and centripetal forces.

        Returns:
        sympy.Matrix
            Coriolis matrix (n x n).
        """
        n = self.dof
        # q = symbols(f'q1:{n+1}')
        # dq = symbols(f'dq1:{n+1}')

        M = self.inertia_matrix
        C = zeros(n, n)
        for i in range(n):
            for j in range(n):
                for k in range(n):
                    c_ijk = 0.5 * (diff(M[i, k], self.q[j]) + diff(M[i, j], self.q[k]) - diff(M[j, k], self.q[i]))
                    C[i, j] += c_ijk * self.dq[k]

        return simplify(C)

    
    def calculate_gravity_vector(self):
        """
        Calculate the gravity vector G(q) by differentiating the total potential energy.

        Returns:
        sympy.Matrix
            Gravity vector.
        """
        from sympy import diff, Matrix

        # Energia potencial total
        U = 0
        for i, com in enumerate(self.r_cis_global):
            U += self.masses[i] * self.g_vec.dot(com)

        # Derivada da energia potencial em relação a cada coordenada q_i
        G = Matrix([diff(U, q_i) for q_i in self.q])

        return G
    
    def compute_inverse_dynamics(self):
        self.q_d = [Function(f'q_{i+1}d')(self.t) for i in range(self.dof)]
        self.dq_d = [qi.diff(self.t) for qi in self.q_d]
        self.ddq_d = [dqi.diff(self.t) for dqi in self.dq_d]

        M = self.inertia_matrix
        C = self.coriolis_matrix
        G = self.gravity_vector
        tau = M.subs(zip(self.q,self.q_d)) @ Matrix(self.ddq_d) + \
              C.subs(zip(self.q,self.q_d)).subs(zip(self.dq, self.dq_d)) @ Matrix(self.dq_d) + \
              G.subs(zip(self.q,self.q_d))
        return simplify(tau)

    def eval_dinamics(self, tau, q_d=None, dq_d=None, ddq_d=None):
        # Importante manter esta ordem!
        if ddq_d: tau = tau.subs(zip(self.ddq_d, ddq_d))
        if dq_d: tau = tau.subs(zip(self.dq_d, dq_d))
        if q_d: tau = tau.subs(zip(self.q_d, q_d))
        return tau.simplify()

In [3]:
# == Define Configuration == #
configuration = 'RRP'
dof = len(configuration)


# == Define Masses == #
masses = Matrix(symbols(f'm1:{dof}'))   # Default: [m1, ..., m{n}]
# m1, m2, m3 = symbols('m1 m2 m3')      # Equivalent to the line above
# masses = Matrix([m1, m2, m3])         # Equivalent to the line above
masses = [10, 20, 30]                   # Example


# == Define inertias == #
inertias = [diag(*[symbols(f'I{i+1}_{k+1}_{k+1}') for k in range(3)]) for i in range(dof)]      # Default: [diag(I111, I122, I133), ..., diag(I{n}11, I{n}22, I{n}33)]
inertias = [symbols('I1')*diag(1,0,0), symbols('I2')*diag(1,1,0), symbols('I3')*diag(1,1,0)]    # Example


# == Define Center of Mass (local) == #
r_cis_symbols = [symbols(f'r_{i+1}{a}') for i in range(dof) for a in {'x','y', 'z'}]
r_cis = [Matrix(r_cis_symbols[i:i+3]) for i in range(0,3*dof,3)]    # Default: [[r1x, r1y, r1z], ..., [r{n}x, r{n}y, r{n}z]]
r_cis = Matrix([[10, 20, 0], [50, 60, 0], [1, 15, 0]])              # Example


# == Define gravity vector (base) == #
g_vec = Matrix([0, 0, -symbols('g')])   # Default: [0, 0, -g]
g_vec = Matrix([0, 0, -9.81])           # Example


# == Define DH parameters == #
q1, q2, q3 = symbols('q1 q2 q3')
L1, L2 = symbols('L1 L2')

# Novo
dh_table = Matrix(
    [[q1, L1, 0, pi/2],
    [q2, 0, 0, pi/2],
    [pi/2, q3+L2, 0, 0]]
)

# Antigo
# dh_table = Matrix(
#     [[q1, L1, 0, pi/2],
#     [q2, L2, 0, -pi/2],
#     [0, q3, 0, 0]]
# )

In [4]:
# == Creating the Robot == #
# r = Robot(configuration='RRP', dh_table=dh_table)                       # Only default values (automatically set inside Class)
r = Robot(configuration='RRP', dh_table=dh_table, inertias=inertias)    # Numeric parameters as you wish: masses=masses, r_cis=r_cis, g_vec=g_vec, inertias=inertias
r.joints

,type,theta,d,a,alpha
1,R,q1(t),L1,0,pi/2
2,R,q2(t),0,0,pi/2
3,P,pi/2,L2 + q3(t),0,0


In [5]:
# DH Parameters
r.thetas, r.ds, r._as_, r.alphas

⎛                            ⎡π⎤⎞
⎜⎡q₁(t)⎤                     ⎢─⎥⎟
⎜⎢     ⎥  ⎡    L₁    ⎤  ⎡0⎤  ⎢2⎥⎟
⎜⎢q₂(t)⎥  ⎢          ⎥  ⎢ ⎥  ⎢ ⎥⎟
⎜⎢     ⎥, ⎢    0     ⎥, ⎢0⎥, ⎢π⎥⎟
⎜⎢  π  ⎥  ⎢          ⎥  ⎢ ⎥  ⎢─⎥⎟
⎜⎢  ─  ⎥  ⎣L₂ + q₃(t)⎦  ⎣0⎦  ⎢2⎥⎟
⎜⎣  2  ⎦                     ⎢ ⎥⎟
⎝                            ⎣0⎦⎠

In [6]:
# DOF, q, dq, ddq, 
r.dof, r.q, r.dq, r.ddq

⎛                                                             ⎡ 2           2  ↪
⎜                          ⎡d          d          d        ⎤  ⎢d           d   ↪
⎜3, [q₁(t), q₂(t), q₃(t)], ⎢──(q₁(t)), ──(q₂(t)), ──(q₃(t))⎥, ⎢───(q₁(t)), ─── ↪
⎜                          ⎣dt         dt         dt       ⎦  ⎢  2           2 ↪
⎝                                                             ⎣dt          dt  ↪

↪           2        ⎤⎞
↪          d         ⎥⎟
↪ (q₂(t)), ───(q₃(t))⎥⎟
↪            2       ⎥⎟
↪          dt        ⎦⎠

In [7]:
# Masses, Inertias, gravity vector
r.masses, r.inertias, r.g_vec

⎛⎡m₁⎤  ⎡⎡I₁  0  0⎤  ⎡I₂  0   0⎤  ⎡I₃  0   0⎤⎤  ⎡0 ⎤⎞
⎜⎢  ⎥  ⎢⎢        ⎥  ⎢         ⎥  ⎢         ⎥⎥  ⎢  ⎥⎟
⎜⎢m₂⎥, ⎢⎢0   0  0⎥, ⎢0   I₂  0⎥, ⎢0   I₃  0⎥⎥, ⎢0 ⎥⎟
⎜⎢  ⎥  ⎢⎢        ⎥  ⎢         ⎥  ⎢         ⎥⎥  ⎢  ⎥⎟
⎝⎣m₃⎦  ⎣⎣0   0  0⎦  ⎣0   0   0⎦  ⎣0   0   0⎦⎦  ⎣-g⎦⎠

In [8]:
# Position of center of mass (local and global)
r.r_cis_local, r.r_cis_global

⎛⎡⎡r_1y⎤  ⎡r_2y⎤  ⎡r_3y⎤⎤  ⎡⎡r_1y⋅cos(q₁(t)) + r_1z⋅sin(q₁(t))⎤  ⎡r₂ₓ⋅sin(q₁(t ↪
⎜⎢⎢    ⎥  ⎢    ⎥  ⎢    ⎥⎥  ⎢⎢                                 ⎥  ⎢             ↪
⎜⎢⎢r₁ₓ ⎥, ⎢r₂ₓ ⎥, ⎢r₃ₓ ⎥⎥, ⎢⎢r_1y⋅sin(q₁(t)) - r_1z⋅cos(q₁(t))⎥, ⎢-r₂ₓ⋅cos(q₁( ↪
⎜⎢⎢    ⎥  ⎢    ⎥  ⎢    ⎥⎥  ⎢⎢                                 ⎥  ⎢             ↪
⎝⎣⎣r_1z⎦  ⎣r_2z⎦  ⎣r_3z⎦⎦  ⎣⎣            L₁ + r₁ₓ             ⎦  ⎣             ↪

↪ )) + r_2y⋅cos(q₁(t))⋅cos(q₂(t)) + r_2z⋅sin(q₂(t))⋅cos(q₁(t)) ⎤  ⎡-r₃ₓ⋅cos(q₁ ↪
↪                                                              ⎥  ⎢            ↪
↪ t)) + r_2y⋅sin(q₁(t))⋅cos(q₂(t)) + r_2z⋅sin(q₁(t))⋅sin(q₂(t))⎥, ⎢-r₃ₓ⋅sin(q₁ ↪
↪                                                              ⎥  ⎢            ↪
↪      L₁ + r_2y⋅sin(q₂(t)) - r_2z⋅cos(q₂(t))                  ⎦  ⎣            ↪

↪ (t))⋅cos(q₂(t)) + r_3y⋅sin(q₁(t)) + r_3z⋅sin(q₂(t))⋅cos(q₁(t)) + (L₂ + q₃(t) ↪
↪                                                                              ↪
↪ (t))⋅cos(q₂(t)) - r_3y⋅c

In [9]:
# Transition Matrix: i-1 -> i
r.dh_matrices

⎡⎡cos(q₁(t))  0  sin(q₁(t))   0 ⎤  ⎡cos(q₂(t))  0  sin(q₂(t))   0⎤  ⎡0  -1  0  ↪
⎢⎢                              ⎥  ⎢                             ⎥  ⎢          ↪
⎢⎢sin(q₁(t))  0  -cos(q₁(t))  0 ⎥  ⎢sin(q₂(t))  0  -cos(q₂(t))  0⎥  ⎢1  0   0  ↪
⎢⎢                              ⎥, ⎢                             ⎥, ⎢          ↪
⎢⎢    0       1       0       L₁⎥  ⎢    0       1       0       0⎥  ⎢0  0   1  ↪
⎢⎢                              ⎥  ⎢                             ⎥  ⎢          ↪
⎣⎣    0       0       0       1 ⎦  ⎣    0       0       0       1⎦  ⎣0  0   0  ↪

↪      0     ⎤⎤
↪            ⎥⎥
↪      0     ⎥⎥
↪            ⎥⎥
↪  L₂ + q₃(t)⎥⎥
↪            ⎥⎥
↪      1     ⎦⎦

In [10]:
# Transition Matrix: 0 -> i
r.base_to_joint

⎡⎡cos(q₁(t))  0  sin(q₁(t))   0 ⎤  ⎡cos(q₁(t))⋅cos(q₂(t))  sin(q₁(t))   sin(q₂ ↪
⎢⎢                              ⎥  ⎢                                           ↪
⎢⎢sin(q₁(t))  0  -cos(q₁(t))  0 ⎥  ⎢sin(q₁(t))⋅cos(q₂(t))  -cos(q₁(t))  sin(q₁ ↪
⎢⎢                              ⎥, ⎢                                           ↪
⎢⎢    0       1       0       L₁⎥  ⎢     sin(q₂(t))             0            - ↪
⎢⎢                              ⎥  ⎢                                           ↪
⎣⎣    0       0       0       1 ⎦  ⎣          0                 0              ↪

↪ (t))⋅cos(q₁(t))  0 ⎤  ⎡sin(q₁(t))   -cos(q₁(t))⋅cos(q₂(t))  sin(q₂(t))⋅cos(q ↪
↪                    ⎥  ⎢                                                      ↪
↪ (t))⋅sin(q₂(t))  0 ⎥  ⎢-cos(q₁(t))  -sin(q₁(t))⋅cos(q₂(t))  sin(q₁(t))⋅sin(q ↪
↪                    ⎥, ⎢                                                      ↪
↪ cos(q₂(t))       L₁⎥  ⎢     0            -sin(q₂(t))             -cos(q₂(t)) ↪
↪                    ⎥  ⎢  

In [11]:
# Transition Matrix: 0 -> n (dof) [from base to end effector]
r.base_to_end_effector

⎡sin(q₁(t))   -cos(q₁(t))⋅cos(q₂(t))  sin(q₂(t))⋅cos(q₁(t))  (L₂ + q₃(t))⋅sin( ↪
⎢                                                                              ↪
⎢-cos(q₁(t))  -sin(q₁(t))⋅cos(q₂(t))  sin(q₁(t))⋅sin(q₂(t))  (L₂ + q₃(t))⋅sin( ↪
⎢                                                                              ↪
⎢     0            -sin(q₂(t))             -cos(q₂(t))          L₁ - (L₂ + q₃( ↪
⎢                                                                              ↪
⎣     0                 0                       0                            1 ↪

↪ q₂(t))⋅cos(q₁(t))⎤
↪                  ⎥
↪ q₁(t))⋅sin(q₂(t))⎥
↪                  ⎥
↪ t))⋅cos(q₂(t))   ⎥
↪                  ⎥
↪                  ⎦

In [12]:
# Jacobian (end effector)
r.jacobian

⎡(-L₂ - q₃(t))⋅sin(q₁(t))⋅sin(q₂(t))  (L₂ + q₃(t))⋅cos(q₁(t))⋅cos(q₂(t))  sin( ↪
⎢                                                                              ↪
⎢(L₂ + q₃(t))⋅sin(q₂(t))⋅cos(q₁(t))   (L₂ + q₃(t))⋅sin(q₁(t))⋅cos(q₂(t))  sin( ↪
⎢                                                                              ↪
⎢                 0                        (L₂ + q₃(t))⋅sin(q₂(t))             ↪
⎢                                                                              ↪
⎢                 0                               sin(q₁(t))                   ↪
⎢                                                                              ↪
⎢                 0                              -cos(q₁(t))                   ↪
⎢                                                                              ↪
⎣                 1                                   0                        ↪

↪ q₂(t))⋅cos(q₁(t))⎤
↪                  ⎥
↪ q₁(t))⋅sin(q₂(t))⎥
↪                  ⎥
↪  -cos(q₂(t))     ⎥
↪  

In [13]:
# Jacobian for each center of mass
r.jacobian_ci

⎡⎡-r_1y⋅sin(q₁(t)) + r_1z⋅cos(q₁(t))  0  0⎤  ⎡r₂ₓ⋅cos(q₁(t)) - r_2y⋅sin(q₁(t)) ↪
⎢⎢                                        ⎥  ⎢                                 ↪
⎢⎢r_1y⋅cos(q₁(t)) + r_1z⋅sin(q₁(t))   0  0⎥  ⎢r₂ₓ⋅sin(q₁(t)) + r_2y⋅cos(q₁(t)) ↪
⎢⎢                                        ⎥  ⎢                                 ↪
⎢⎢                0                   0  0⎥  ⎢                                 ↪
⎢⎢                                        ⎥, ⎢                                 ↪
⎢⎢                0                   0  0⎥  ⎢                                 ↪
⎢⎢                                        ⎥  ⎢                                 ↪
⎢⎢                0                   0  0⎥  ⎢                                 ↪
⎢⎢                                        ⎥  ⎢                                 ↪
⎣⎣                1                   0  0⎦  ⎣                                 ↪

↪ ⋅cos(q₂(t)) - r_2z⋅sin(q₁(t))⋅sin(q₂(t))  (-r_2y⋅sin(q₂(t)) + r_2z⋅cos(q₂(t) ↪
↪                          

In [14]:
# Inertia Matrix
r.inertia_matrix

⎡                        ⎛           2                    2                    ↪
⎢   ⎛    2       2⎞      ⎜   2   r_2y ⋅cos(2⋅q₂(t))   r_2y                     ↪
⎢m₁⋅⎝r_1y  + r_1z ⎠ + m₂⋅⎜r₂ₓ  + ────────────────── + ───── + r_2y⋅r_2z⋅sin(2⋅ ↪
⎢                        ⎝               2              2                      ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎣                                                                              ↪

↪              2                    2⎞      ⎛    2                  2          ↪
↪          r_2z ⋅cos(2⋅q₂(t))   r_2z ⎟      ⎜  L₂ ⋅cos(2⋅q₂(t))   L₂           ↪
↪ q₂(t)) - ────────────────── + ─────⎟ + m₃⋅⎜- ──────────────── + ─── - L₂⋅r₃ₓ ↪
↪                  2       

In [15]:
# Coriolis/Centripetal
r.coriolis_matrix

⎡                                                                              ↪
⎢- 0.5⋅m₃⋅(L₂⋅cos(2⋅q₂(t)) - L₂ + r₃ₓ⋅sin(2⋅q₂(t)) + r_3z⋅cos(2⋅q₂(t)) - r_3z  ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                ⎛   ⎛      2  ↪
⎢                                                           -0.5⋅⎝m₂⋅⎝- r_2y ⋅ ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎣                                                                              ↪

↪                               d               ⎛   ⎛      2                   ↪
↪ + q₃(t)⋅cos(2⋅q₂(t)) - q₃

In [16]:
# Gravity vector
r.gravity_vector

⎡                                                      0                       ↪
⎢                                                                              ↪
⎢-g⋅m₂⋅(r_2y⋅cos(q₂(t)) + r_2z⋅sin(q₂(t))) - g⋅m₃⋅(-r₃ₓ⋅cos(q₂(t)) + r_3z⋅sin( ↪
⎢                                                                              ↪
⎣                                               g⋅m₃⋅cos(q₂(t))                ↪

↪                                  ⎤
↪                                  ⎥
↪ q₂(t)) + (L₂ + q₃(t))⋅sin(q₂(t)))⎥
↪                                  ⎥
↪                                  ⎦

In [17]:
# Torques and Forces (Tau)
r.torques

⎡                                                                              ↪
⎢      2                   d           d                   2                   ↪
⎢1.0⋅L₂ ⋅m₃⋅sin(2⋅q_2d(t))⋅──(q_1d(t))⋅──(q_2d(t)) - 0.5⋅L₂ ⋅m₃⋅cos(2⋅q_2d(t)) ↪
⎢                          dt          dt                                      ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                           

In [18]:
# See which symbols we can substitute
print('Da pra substituir esses símbolos:')
print(r.torques.free_symbols)
print(f'\nIgual, mas bonito (e em outra ordem):')
r.torques.free_symbols

Da pra substituir esses símbolos:
{m1, r_1y, r_2x, r_3x, r_3y, m3, r_3z, I2, r_2y, I3, r_2z, g, r_1z, t, m2, L2}

Igual, mas bonito (e em outra ordem):


{I₂, I₃, L₂, g, m₁, m₂, m₃, r_1y, r_1z, r₂ₓ, r_2y, r_2z, r₃ₓ, r_3y, r_3z, t}

In [19]:
# Substitution and evaluation:

# Define what to substitute
substitution_dict = {
    'm1':2, 'm2':1, 'm3':1,
    'I1':50, 'I2': 10, 'I3':300,
    'r_1x':1.5, 'r_1y':0, 'r_1z':0,
    'r_2x':1.5, 'r_2y':0, 'r_2z':0,
    'r_3x':1.5, 'r_3y':0, 'r_3z':0,
    'L1':3, 'L2':4,
    'g':9.81
}

# Choose a function or expression
f = r.torques

# Options (basicamente equivalentes)
tau = f.subs(substitution_dict).simplify()        # For substitution (can be other symbolic expression, i.e.)
# f.evalf(subs=substitution_dict).simplify()  # For numeric substitution and evaluation
tau

⎡                                                                              ↪
⎢        2                   d           d                     2               ↪
⎢1.0⋅q_3d (t)⋅sin(2⋅q_2d(t))⋅──(q_1d(t))⋅──(q_2d(t)) - 0.5⋅q_3d (t)⋅cos(2⋅q_2d ↪
⎢                            dt          dt                                    ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                           

In [21]:
# To evaluate with the desired values for q, dq and ddq:
q_d = [pi/2, pi/4, 3]
dq_d = [30, 20, 13]
ddq_d = [5, 3, 1]

# Prestar atenção, a ordem de substituição é: ddq, dq, q
#   Se apenas q for setada, dq e ddq são calculados automaticamente, pois são suas derivadas. (i.e., se colocar constantes, dq e ddq são ZERO)
#   Se apenas dq for setada, ddq é calculada automaticamente, pois é sua derivada. (q não sofre alteração)
#   Se todos os valores forem setados, segue-se a ordem de substituição, para que o sistema não identifique automaticamente os valores das derivadas e considere os valores dados.
#   Tomar cuidado quando for substituir.
# Interessante: podemos colocar uma trajetória desejada em função do tempo para seguir q_d(t), que os cálculos consideram as derivadas automaticamente: ver Exemplo 2
# Exemplo 1
r.eval_dinamics(tau=tau, q_d=q_d, dq_d=dq_d, ddq_d=ddq_d)

# Exemplo 2: 
# Trajetória plano X-Y [r.t, exp(-r.t), 0] (para substituir valores de t, basta: <res>.subs('t', valor) ou <res>.subs(r.t, valor))
r.eval_dinamics(tau=tau, q_d=[r.t, exp(-r.t), 0]).subs('t', 4)   # .subs('t', 4)

⎡                            -4    ⎛   -4⎞         -4    ⎛   -4⎞               ↪
⎢                   - 13.75⋅ℯ  ⋅sin⎝2⋅ℯ  ⎠ + 12.0⋅ℯ  ⋅cos⎝2⋅ℯ  ⎠               ↪
⎢                                                                              ↪
⎢           ⎛ -4⎞            ⎛   -4⎞          ⎛   -4⎞           -4             ↪
⎢- 39.24⋅sin⎝ℯ  ⎠ - 6.875⋅sin⎝2⋅ℯ  ⎠ + 6.0⋅cos⎝2⋅ℯ  ⎠ + 328.25⋅ℯ   + 14.715⋅co ↪
⎢                                                                              ↪
⎢                -8           ⎛   -4⎞        -4          ⎛   -4⎞           ⎛ - ↪
⎣    -2.0 - 4.0⋅ℯ   + 0.75⋅sin⎝2⋅ℯ  ⎠ + 1.5⋅ℯ   + 2.0⋅cos⎝2⋅ℯ  ⎠ + 9.81⋅cos⎝ℯ  ↪

↪       ⎤
↪       ⎥
↪       ⎥
↪  ⎛ -4⎞⎥
↪ s⎝ℯ  ⎠⎥
↪       ⎥
↪ 4⎞    ⎥
↪  ⎠    ⎦

In [22]:
# Alternativamente, para substituir valores para t quando só estiver o t faltando:
new_tau = r.eval_dinamics(tau=tau, q_d=[r.t, exp(-r.t), 0])
f = lambdify('t', new_tau)
f(t=4)

array([[ 0.21041706],
       [25.75016606],
       [ 9.86061182]])